# Config

In [61]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises
import plotly
import plotly.express as px
from munkres import Munkres, print_matrix
import plotly.graph_objects as go


In [62]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 11
weekEnd = 11
def_starting_pts = 18
pr_bonus = 0
kr_bonus = 0
username = 'dirtywizard'

# Extract

In [63]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
preds = preds.loc[(preds['week']>=weekStart)&(preds['week']<=weekEnd)]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [64]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [65]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [66]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get league info
user_league = Leagues.get_league(league_id)

# Get roster positions 
rosterList = user_league['roster_positions']
# Sort rosterList
sort_order = ['QB', 'RB', 'WR', 'TE', 'K', 'DEF', 'FLEX', 'SUPER_FLEX', 'BN']
rosterList.sort(key=sort_order.index)

# Get number of franchises
number_of_franchises = user_league['total_rosters']

# Transform

In [67]:
# Customize predictions for league
# Calculate projected points according to that league's scoring settings
# Multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + def_starting_pts
# Add extras for Punt/kick returners
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Merge in point projections with franchise data
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [68]:
full.head()

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,KR,date_prediction,pts_proj,owner_id,taxi,starters,players,user_id,franchise_name,display_name
0,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],3.0,...,NaN,20231116,4.705,905172334586535936,NaN,NaN,True,905172334586535936,kevinbash,kevinbash
1,2251,Logan Thomas,1991-07-01,32.0,250.0,78,WAS,TE,['TE'],1.0,...,NaN,20231116,4.925,904381167934095360,NaN,True,True,904381167934095360,GusTheBus,mdwglobe
2,7922,Riley Patterson,1999-09-07,24.0,195.0,72,DET,PK,['K'],1.0,...,NaN,20231116,9.900,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff
3,6865,Colby Parkinson,1999-01-08,24.0,265.0,79,SEA,TE,['TE'],2.0,...,NaN,20231116,2.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MIN,MIN,NaN,NaN,NaN,NaN,MIN,DEF,['DEF'],NaN,...,NaN,20231116,4.710,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15


# Create cost array

# Munkres

In [69]:
# Initialize munkdf
optimized = full.copy()
valProjs = pd.DataFrame()
rosterDict = {
    'QB':['QB'], 
    'RB':['RB'],
    'WR':['WR'],
    'TE':['TE'],
    'K':['PK'],
    'DEF':['DEF'],
    'FLEX':['RB', 'WR', 'TE'],
    'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
}
df = optimized[['player_id', 'week', 'position', 'pts_proj', 'franchise_name']]
# Select only rostered players
df = df.loc[df['franchise_name'].notna()]
# Create a column for each possible roster spot
counter = 1
for spot in rosterList:
    if spot != "BN":
        # Select players who could have value in that role and fill their points under that column
        df.loc[df['position'].isin(rosterDict[spot]), f'{counter}_{spot}'] = df.loc[df['position'].isin(rosterDict[spot]), 'pts_proj']
        counter += 1
df = df.fillna(0)

# Loop through franchises and weeks
for thisWeek in df['week'].unique():
    for thisFranchise in df.loc[df['franchise_name'].notna()]['franchise_name'].unique():
        munkdf = df.loc[(df['week']==thisWeek) & (df['franchise_name']==thisFranchise)].reset_index(drop=True)
        array = munkdf.drop(columns=['player_id', 'week','position','pts_proj','franchise_name'])

        matrix = array.values
        cost_matrix = []
        for row in matrix:
            cost_row = []
            for col in row:
                # Subtract from maximum possible points to find cost matrix
                cost_row += [array.max().max() - col]
            cost_matrix += [cost_row]

        m = Munkres()
        indexes = m.compute(cost_matrix)
        # print_matrix(matrix, msg='Highest profit through this matrix:')
        # total = 0
        # for row, column in indexes:
        #     value = matrix[row][column]
        #     total += value
        #     print(f'({row}, {column}) -> {value}')

        # print(f'total profit={total}')
        # Set column for values only positive when that player would be played
        munkdf.loc[munkdf.index.isin([x[0] for x in indexes]), 'val_proj'] = munkdf.loc[munkdf.index.isin([x[0] for x in indexes]), 'pts_proj']
        munkdf = munkdf[['player_id', 'week','val_proj']]
        # Append to value holder
        valProjs = pd.concat([valProjs, munkdf], axis=0, ignore_index=True)

In [70]:
optimized = optimized.merge(valProjs, how='left', on=['player_id', 'week'])
optimized

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,date_prediction,pts_proj,owner_id,taxi,starters,players,user_id,franchise_name,display_name,val_proj
0,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],3.0,...,20231116,4.705,905172334586535936,NaN,NaN,True,905172334586535936,kevinbash,kevinbash,NaN
1,2251,Logan Thomas,1991-07-01,32.0,250.0,78,WAS,TE,['TE'],1.0,...,20231116,4.925,904381167934095360,NaN,True,True,904381167934095360,GusTheBus,mdwglobe,4.925
2,7922,Riley Patterson,1999-09-07,24.0,195.0,72,DET,PK,['K'],1.0,...,20231116,9.900,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,9.900
3,6865,Colby Parkinson,1999-01-08,24.0,265.0,79,SEA,TE,['TE'],2.0,...,20231116,2.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MIN,MIN,NaN,NaN,NaN,NaN,MIN,DEF,['DEF'],NaN,...,20231116,4.710,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15,4.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,6783,Jerry Jeudy,1999-04-24,24.0,193.0,73,DEN,WR,['WR'],2.0,...,20231116,9.350,904398822170636288,NaN,NaN,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,NaN
430,5038,Michael Gallup,1996-03-04,27.0,198.0,73,DAL,WR,['WR'],3.0,...,20231116,4.428,904399374891855872,NaN,NaN,True,904399374891855872,Comeback 👑’s,comebackking15,NaN
431,6786,CeeDee Lamb,1999-04-08,24.0,200.0,74,DAL,WR,['WR'],1.0,...,20231116,14.660,342869377223385088,NaN,True,True,342869377223385088,*New Name Pending*,nrector,14.660
432,6918,Salvon Ahmed,1998-12-29,24.0,197.0,71,MIA,RB,['RB'],4.0,...,20231116,2.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
optimized.groupby('franchise_name')['val_proj'].sum().sort_values(ascending=False)

franchise_name
Saskatoon Squatches         180.8598
Croccity Body Snatchers     172.5338
kevinbash                   164.9573
Verdanks Vacqueros FFC      156.7380
Pretty Big Wieners          152.8012
Comeback 👑’s                151.1720
DangeRUSS Last Ride         148.6456
📜 Providence 🪬 Spirits 🥂    148.1406
2014champ                   141.1769
GusTheBus                   129.9763
*New Name Pending*          127.6729
Idk much about soccer       117.0070
Name: val_proj, dtype: float64

In [72]:
def getSeasonTotals(df):
    # Sum up pts_proj and val_proj
    totals_proj = df.groupby('player_id')['pts_proj'].sum().reset_index()
    totals_rel = df.groupby('player_id')['val_proj'].sum().reset_index()
    # Merge in totals with player metadata
    unis = df.drop_duplicates(subset='player_id', ignore_index=True).drop(columns=['pts_proj', 'val_proj'])
    df = unis.merge(totals_proj, how='left', on='player_id').merge(totals_rel, how='left', on='player_id')
    return df

In [73]:
def compareFranchises(df, how):
    '''
    df: dataframe;
    how: 'relative' uses relative point values, 'absolute' uses actual point projections
    '''
    if how=='relative':
        main_stat = 'val_proj'
        second_stat = 'pts_proj'
    elif how=='absolute':
        main_stat = 'pts_proj'
        second_stat = 'val_proj'        
    dfs = df.groupby('franchise_name').sum().round(0)
    fig = px.bar(df, 
                x="franchise_name", 
                y=main_stat, 
                color="position", 
                text='full_name', 
                color_discrete_map={
                    # "QB": "hsla(210, 60%, 25%, 1)", 
                    # "RB": "hsla(12, 50%, 45%, 1)", 
                    # "WR": "hsla(267, 40%, 45%, 1)", 
                    # "TE": "hsla(177, 68%, 36%, 1)", 
                    # "PK": "hsla(14, 30%, 40%, 1)", 
                    # "DEF": "hsla(35, 70%, 65%, 1)"}, 
                    "QB": "#591D73", #
                    "RB": "#233257", 
                    "WR": "#4068B8", 
                    "TE": "#AEDFF2", #529AD9
                    "PK": "#B3BF54", 
                    "DEF": "#667825"}, 
                category_orders={
                    "pos": ["QB", "RB", "WR", "TE", "PK", "DEF"]},
                hover_name="full_name",
                hover_data={
                    main_stat:True, second_stat:True,
                    'full_name':False, 'position':False, 'franchise_name':False
                    },
                labels={
                    "franchise_name":"Franchise",
                    "val_proj":"Relative Value",
                    'pts_proj':"Predicted Points",
                }
                )
    fig.update_traces(
        textposition='inside',
    )
    fig.update_yaxes(
        visible=False
    )
    fig.add_trace(go.Scatter(
                x=dfs.index, 
                y=dfs[main_stat],
                text=dfs[main_stat],
                mode='text',
                textposition='top center',
                textfont=dict(
                    size=12,
                ),
                showlegend=False
            ))
    fig.update_layout(
                barmode='stack', 
                #xaxis={'categoryorder':'sum ascending'},
                plot_bgcolor='rgba(0,0,0,0)',
                title="Franchise Comparison",
                font_family="Input Serif",
                #font_family="Skia",
                showlegend=False,
                xaxis_title=None
                )
    # fig.update_xaxes(
    #     categoryorder='sum ascending'
    # )
    return fig

In [74]:
# Sum points across the season
optimized = getSeasonTotals(optimized)
# Visualize
fig = compareFranchises(optimized, how='relative')
fig.show()

In [60]:
optimized

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,date_prediction,owner_id,taxi,starters,players,user_id,franchise_name,display_name,pts_proj,val_proj
0,4234,Noah Brown,1996-01-06,27.0,215.0,74,HOU,WR,['WR'],NaN,...,20231010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.940,0.000
1,2251,Logan Thomas,1991-07-01,32.0,250.0,78,WAS,TE,['TE'],1.0,...,20231010,904381167934095360,NaN,True,True,904381167934095360,GusTheBus,mdwglobe,60.445,44.925
2,7922,Riley Patterson,1999-09-07,24.0,195.0,72,DET,PK,['K'],1.0,...,20231010,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,90.020,90.020
3,10216,Kenny McIntosh,2000-03-03,23.0,210.0,72,SEA,RB,['RB'],NaN,...,20231010,208058775418961920,NaN,NaN,True,208058775418961920,📜 Providence 🪬 Spirits 🥂,ethrec,9.639,0.906
4,6865,Colby Parkinson,1999-01-08,24.0,251.0,79,SEA,TE,['TE'],2.0,...,20231010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.885,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,6783,Jerry Jeudy,1999-04-24,24.0,193.0,73,DEN,WR,['WR'],1.0,...,20231010,904398822170636288,NaN,NaN,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,91.125,40.575
504,5038,Michael Gallup,1996-03-04,27.0,198.0,73,DAL,WR,['WR'],1.0,...,20231010,904399374891855872,NaN,NaN,True,904399374891855872,Comeback 👑’s,comebackking15,55.787,19.896
505,6786,CeeDee Lamb,1999-04-08,24.0,200.0,74,DAL,WR,['WR'],1.0,...,20231010,342869377223385088,NaN,True,True,342869377223385088,*New Name Pending*,nrector,137.800,137.800
506,6918,Salvon Ahmed,1998-12-30,24.0,197.0,71,MIA,RB,['RB'],3.0,...,20231010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.355,0.000


In [76]:
optimized.loc[optimized['franchise_name']=='Croccity Body Snatchers'].sort_values('pts_proj')

,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,date_prediction,owner_id,taxi,starters,players,user_id,franchise_name,display_name,pts_proj,val_proj
50,DEN,DEN,NaN,NaN,NaN,NaN,DEN,DEF,['DEF'],NaN,...,20231116,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,-0.1900,0.0000
82,9222,Zach Evans,2001-05-30,22.0,202.0,71,LAR,RB,['RB'],3.0,...,20231116,725907041763303424,True,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,0.8250,0.0000
167,8181,Connor Heyward,1999-01-22,24.0,230.0,72,PIT,TE,['TE'],2.0,...,20231116,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,1.9200,0.0000
106,SF,SF,NaN,NaN,NaN,NaN,SF,DEF,['DEF'],NaN,...,20231116,725907041763303424,NaN,True,True,725907041763303424,Croccity Body Snatchers,dirtywizard,2.6363,2.6363
301,3271,Tyler Higbee,1993-01-01,30.0,255.0,78,LAR,TE,['TE'],1.0,...,20231116,725907041763303424,NaN,True,True,725907041763303424,Croccity Body Snatchers,dirtywizard,4.7360,4.7360
280,8225,Daniel Bellinger,2000-09-22,23.0,255.0,78,NYG,TE,['TE'],1.0,...,20231116,725907041763303424,NaN,True,True,725907041763303424,Croccity Body Snatchers,dirtywizard,4.9260,4.9260
343,5185,Allen Lazard,1995-12-11,27.0,227.0,77,NYJ,WR,['WR'],2.0,...,20231116,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,4.9710,0.0000
348,7066,K.J. Osborn,1997-06-10,26.0,203.0,71,MIN,WR,['WR'],3.0,...,20231116,725907041763303424,NaN,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,5.6020,0.0000
281,WAS,WAS,NaN,NaN,NaN,NaN,WAS,DEF,['DEF'],NaN,...,20231116,725907041763303424,NaN,True,True,725907041763303424,Croccity Body Snatchers,dirtywizard,5.6205,5.6205
345,8259,Cameron Dicker,2000-05-06,23.0,216.0,73,LAC,PK,['K'],1.0,...,20231116,725907041763303424,True,NaN,True,725907041763303424,Croccity Body Snatchers,dirtywizard,8.8500,0.0000
